The next cell just beamforms an image:

In [ ]:
from vbeam.fastmath import backend_manager

backend_manager.active_backend = "jax"

import jax
import numpy as np
import pyuff

from vbeam.beamformers import get_beamformer
from vbeam.data_importers import import_pyuff
from vbeam.scan import Scanlines, sector_scan
from vbeam.util.download import cached_download

# Download and read the channel data
data_url = "http://www.ustb.no/datasets/Verasonics_P2-4_parasternal_long_small.uff"
uff = pyuff.Uff(cached_download(data_url))
channel_data = uff.read("/channel_data")

# Import the data
data = import_pyuff(channel_data, frames=0)

# Define and set a custom sector scan
scan_angles = np.array([wave.source.azimuth for wave in channel_data.sequence])
scan_depths = np.linspace(0e-3, 110e-3, 512)
data.scan = sector_scan(scan_angles, scan_depths).with_points_reshaper(Scanlines())

# Get a default beamformer without cartesian mapping
beamformer = jax.jit(get_beamformer(data, cartesian_map=False))
result = beamformer()

Let's rasterize it!

In [ ]:
import matplotlib.pyplot as plt
from raxterize.core import image2polygons, lerp_all

from vbeam.scan.common import grid

# Convert the image to polygons
polygons = image2polygons(data.scan.get_point_cube(), result)

# Create the grid that we want to rasterize to
nx, ny = 200, 200
new_grid = grid(
    np.linspace(-0.08, 0.08, nx),
    np.array([0]),
    np.linspace(0, 0.11, ny),
    shape=(nx * ny, 3),
)

# Rasterize the polygons to the grid
rasterized_result = lerp_all(polygons, new_grid)

# Show image
plt.imshow(rasterized_result.reshape(nx, ny).T, aspect="auto")
